In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
from tqdm import tqdm

import os

ROOT = r'L:\\basic\\divg\\PATH-COMPAI\\datasets\\Barrett\\Barrett ASL 21-11-22\\'

In [ ]:
# Get brown and blue layers
def get_brown_blue_layers(img):
    brown = np.array([128,  90,  90])
    blue  = np.array([150, 170, 190])
    template = np.ones((img.shape[0]*img.shape[1],3))
    brown_copy = template.copy()
    blue_copy  = template.copy()
    brown_copy[:] = brown
    blue_copy [:] = blue
    pixels = img.reshape(img.shape[0]*img.shape[1],3)
    brown_mask = np.isclose(pixels, brown_copy, atol=50).astype(np.float32).min(axis=-1).reshape(img.shape[0],img.shape[1])
    blue_mask  = np.isclose(pixels, blue_copy , atol=50).astype(np.float32).min(axis=-1).reshape(img.shape[0],img.shape[1])
    return brown_mask, blue_mask

def save_brown_mask(data_path=os.path.join(ROOT, "patch_dataset_s1_512", "p53")):
    patch_paths = [os.path.join(data_path, nm) for nm in os.listdir(data_path)]

    for i, path in tqdm(enumerate(patch_paths)):
        try:
            img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        except:
            print(path)
            continue
        brown_mask, blue_mask = get_brown_blue_layers(img)

        fig, ax = plt.subplots(1,2)
        ax[0].imshow(img)
        ax[0].axis("Off")
        ax[1].imshow(brown_mask, cmap="gray")
        ax[1].axis("Off")
        ax[1].set_title(f"{brown_mask.sum()/(blue_mask.sum()+brown_mask.sum()):.4f}")

        plt.savefig(os.path.join(ROOT, "visualisation", os.path.basename(path)), bbox_inches="tight", 
                    # pad_inches=0
                    )
        plt.close()

In [ ]:
data_path = os.path.join(ROOT, "patch_dataset_s1_512", "p53")
patch_paths = [os.path.join(data_path, nm) for nm in os.listdir(data_path)]
img = cv2.cvtColor(cv2.imread(patch_paths[0]), cv2.COLOR_BGR2RGB)
brown_mask, blue_mask = get_brown_blue_layers(img)

fig, ax = plt.subplots(1,3,figsize=(30,30))
ax[0].imshow(img)
ax[1].imshow(brown_mask)
ax[2].imshow(blue_mask)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data
from skimage.color import rgb2hed, hed2rgb


def get_h_d_layers(ihc_rgb):
    ihc_hed = rgb2hed(ihc_rgb) # Separate the stains from the IHC image

    # Create an RGB image for each of the stains
    null = np.zeros_like(ihc_hed[:, :, 0])
    ihc_h = hed2rgb(np.stack((ihc_hed[:, :, 0], null, null), axis=-1))
    ihc_d = hed2rgb(np.stack((null, null, ihc_hed[:, :, 2]), axis=-1))
    return ihc_h, ihc_d

def plot_h_d_layers(ihc_rgb, show=True):
    ihc_h, ihc_d = get_h_d_layers(ihc_rgb)

    # print((1-ihc_h).sum(), (1-ihc_d).sum())
    DAB_density = (1-ihc_d).sum()/((1-ihc_h).sum()+(1-ihc_d).sum())

    # Display
    fig, axes = plt.subplots(1, 3, figsize=(7, 6), sharex=True, sharey=True)
    ax = axes.ravel()

    ax[0].imshow(ihc_rgb)
    ax[0].set_title("Original image")

    ax[1].imshow(ihc_h)
    ax[1].set_title("Hematoxylin")

    ax[2].imshow(ihc_d)
    ax[2].set_title(f"DAB {DAB_density:.2f}")

    for a in ax.ravel():
        a.axis('off')

    fig.tight_layout()
    if show:
        plt.show()

    return DAB_density


def save_h_d_layers(data_path=os.path.join(ROOT, "patch_dataset_s1_512", "p53")):
    patch_paths = [os.path.join(data_path, nm) for nm in os.listdir(data_path)]

    for i, path in tqdm(enumerate(patch_paths[1163:])):
        try:
            img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        except:
            print(path)
            continue
        DAB_density = plot_h_d_layers(img, show=False)

        # Save plot
        plt.savefig(os.path.join(ROOT, "visualisation", f"{DAB_density:.2f}_{os.path.basename(path)}"), bbox_inches="tight",
                    # pad_inches=0
                    )
        plt.close()
        
# Example IHC image
ihc_rgb = data.immunohistochemistry()
# plot_h_d_layers(ihc_rgb)

In [ ]:
save_h_d_layers()

In [ ]:
images = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in patch_paths[:10]]
for img in images:
    plot_h_d_layers(img)
    plt.show()